In [ ]:
from collections import defaultdict
import os
import pathlib
import pickle
import random
import sys
import numpy as np
import pandas as pd

# random.seed(723)
# pd.set_option("display.max_columns", 1000)
# pd.set_option("display.max_rows", 1000)

CURR = "."
PATH = "english2.csv"  # 読み込む単語帳のパス
CHAPTER_START = 1  # 何章から練習の対象とするか
CHAPTER_END = 1 << 60  # 何章まで練習の対象とするか
REVERSE = False  # Falseなら和訳モード、Trueなら英訳モード
CNT = 15  # 挑戦する回数
LOAD = True  # 前回までの記録を読み込む
RECORD = True  # 結果を記録する  ※和訳モードのみ
PROB = True  # Trueなら過去の正解率に応じて出題確率を制御
LOSS = 3  # 正解数から目減りさせる数 (LOSS回正解するまで出題確率が下がらず、その後もLOSS回分目減りさせる)
MIN_PROB = 0.1  # 全問正解の場合に収束する出題確率
PENALTY = 0  # 不正解時のペナルティ (正解数を減らす)
ALL = False  # 全問出題モード　 ※CNTに関わらず指定chapの全問出題される
RANDOM = True  # ランダム出題モード

curr = pathlib.Path(CURR)
path = pathlib.Path(PATH)

In [ ]:
def load(words, LOAD=LOAD):
    """過去の記録を読み込む
    """
    tmp_path = curr / "results" / (path.stem + "_dict_try.pickle")
    if tmp_path.exists() and LOAD:
        with open(tmp_path, "rb") as f:
            dict_try = pickle.load(f)
    else:
        dict_try = defaultdict(int)
    tmp_path = curr / "results" / (path.stem + "_dict_correct.pickle")
    if tmp_path.exists() and LOAD:
        with open(tmp_path, "rb") as f:
            dict_correct = pickle.load(f)
    else:
        dict_correct = defaultdict(int)
    return cleansing(dict_try, words), cleansing(dict_correct, words)


def cleansing(d, words):
    """単語帳に存在する単語の記録のみ抜粋する
    """
    dict_ret = defaultdict(int)
    for k, v in d.items():
        if k in words:
            dict_ret[k] = v
    return dict_ret


def dump(dict_try, dict_correct):
    """記録を保存する
    """
    with open(curr / "results" / (path.stem + "_dict_try.pickle"), "wb") as f:
        pickle.dump(dict_try, f)
    with open(curr / "results" / (path.stem + "_dict_correct.pickle"), "wb") as f:
        pickle.dump(dict_correct, f)


def custom_input(dict_try, dict_correct):
    """文字列を受け取り色々する
    """
    inp = input()
    if inp in {"end", "えんｄ", "e", "え"}:
        if RECORD:
            dump(dict_try, dict_correct)
        sys.exit()
    else:
        return inp


def probability(df, dict_try, dict_correct, idx):
    """出題確率を制御する
    """
    word = df["word"][idx]
    accuracy = 0 if dict_try[word] == 0 else max(0, dict_correct[word] - LOSS) / dict_try[word]
    return 1 if dict_try[word] == 0 else 1 - accuracy * (1 - MIN_PROB)


def record(dict_try, dict_correct, idx, word):
    """記録するための処理
    """
    print("正解しましたか？ (y/n/覚えた/リセット)\n>>")
    inp = custom_input(dict_try, dict_correct)
    dict_try[word] += 1
    if inp in {"覚えた", "おぼえた"}:
        dict_correct[word] = dict_try[word]
    elif inp == "リセット":
        dict_correct[word] = 0
        dict_try[word] = 0
    elif inp in {"y", "yes", "はい", "正解", "1", "いぇｓ", "ｙ"}:
        dict_correct[word] += 1
    else:
        dict_correct[word] -= PENALTY
    print("\n")


def practice(df, dict_try, dict_correct, i, idx):
    """単語の練習のための処理
    """
    if REVERSE:
        word = df["japanese"][idx]
        print("{}. 「{}」 を英語で言うと？\n>>".format(i + 1, word))
        inp = custom_input(dict_try, dict_correct)
        print("正解は 「{}」\n".format(df["word"][idx]))
    else:
        word = df["word"][idx]
        accuracy = 0 if dict_try[word] == 0 else dict_correct[word] / dict_try[word]
        print("{}. 「{}」 の品詞と日本語訳は？  ※これまでの正解率は{:.1f}%\n>>".format(
            i + 1, word, accuracy * 100))
        inp = custom_input(dict_try, dict_correct)
        print("正解は 「{}」 「{}」\n".format(df["part_of_speech"][idx], df["japanese"][idx]))
        record(dict_try, dict_correct, idx, word)


def get_accuracy():
    """正解率を取得する
    """
    df_raw = pd.read_csv(curr / "dict" / PATH, encoding="utf-8")
    df = df_raw.copy()
    words = set(df["word"].values)
    df = df.loc[(df["chap"] >= CHAPTER_START) & (df["chap"] <= CHAPTER_END)]
    df.reset_index(drop=True, inplace=True)
    dict_try, dict_correct = load(words)

    result = []
    for k, v in dict_try.items():
        if v > 0:
            result.append(((dict_correct[k] * 100) / v, k, str(dict_correct[k]) + "/" + str(v)))
    result.sort(reverse=True)
    for v, k, cnt in result:
        print("{}: {:.1f} %　…… ({})".format(k, v, cnt))


def main():
    global CURR
    global PATH
    global CHAPTER_START
    global CHAPTER_END
    global REVERSE
    global CNT
    global LOAD
    global RECORD
    global PROB
    global LOSS
    global MIN_PROB
    global PENALTY
    global ALL
    global RANDOM
    global curr
    global path

    df_raw = pd.read_csv(curr / "dict" / PATH, encoding="utf-8")
    df = df_raw.copy()
    words = set(df["word"].values)
    df = df.loc[(df["chap"] >= CHAPTER_START) & (df["chap"] <= CHAPTER_END)]
    df.reset_index(drop=True, inplace=True)
    dict_try, dict_correct = load(words)

    nums = df.shape[0]
    idxs = [x for x in range(nums)]

    if RANDOM:
        random.shuffle(idxs)

    if ALL:
        CNT = nums

    i = 0
    j = 0

    while i < CNT:
        idx = idxs[(i + j) % nums]
        if not ALL and PROB:
            if probability(df, dict_try, dict_correct, idx) < random.uniform(0, 1):
                j += 1
                continue
        practice(df, dict_try, dict_correct, i, idx)
        i += 1
    else:
        if RECORD:
            dump(dict_try, dict_correct)
        print("{}問終わり！".format(i))


if __name__ == '__main__':
    main()

In [ ]:
get_accuracy()